In [ ]:
import cf_units
import datetime
import iris
import iris.analysis.calculus as icalc
import numpy as np
import os
#
import umtools.irismode as umu
from umtools.utils import grdstep

In [ ]:
iris.FUTURE.netcdf_promote = True
um_res = grdstep('km2p2')

In [ ]:
fcst_init = '25_1200'

In [ ]:
f = iris.load(os.path.join(mypaths.um_dir,um_res.key,fcst_init,'model_levels_full/processed/umnsa_rlev_obs_11-14h_024-proc.nc'))

In [ ]:
def cube_georef_to_cart(cube, dx=1, dy=None, remove_z_bounds=True):
    if dy is None:
        dy = dx
    xpoints = np.array(range(u_orig.coord('grid_longitude').shape[0]))*dx
    ypoints = np.array(range(u_orig.coord('grid_latitude').shape[0]))*dy
    
    xcoord = iris.coords.DimCoord(points=xpoints, standard_name='longitude', units=cf_units.Unit('m'))
    ycoord = iris.coords.DimCoord(points=ypoints, standard_name='latitude', units=cf_units.Unit('m'))
    
    zcoord = iris.coords.DimCoord.from_coord(u_orig.coord('atmosphere_hybrid_height_coordinate'))
    if remove_z_bounds:
        zcoord.bounds = None
    
    res = iris.cube.Cube(data=cube.data, standard_name=cube.name(), units=cube.units)
    
    res.add_dim_coord(zcoord,0)
    res.add_dim_coord(ycoord,1)
    res.add_dim_coord(xcoord,2)
    
    return res

In [ ]:
u_orig = f.extract_strict('x_wind')
v_orig = f.extract_strict('y_wind')
w_orig = f.extract_strict('upward_air_velocity')

In [ ]:
u = cube_georef_to_cart(u_orig, dx=um_res.to_flt())
v = cube_georef_to_cart(v_orig, dx=um_res.to_flt())
w = cube_georef_to_cart(w_orig, dx=um_res.to_flt())
w.rename('z_wind')

In [ ]:
a = icalc.curl(u,v,w)